In [ ]:
import random
import numpy as np
class Solution:
    def __init__(self,populars,vector=None):
        if vector: self.vector=vector
        else: vector=self.get_vector()
        self.populars=populars
        self.not_populars = [i for i in range(len(vector)) if i==0]
        #self.populars_total=len(self.populars)
    
    @property
    def Popular_size(self):
        return len(self.populars)
    
    def get_vector(self):
        vector=np.zeros()
        for i in self.populars:
            vector[i]=1
        return vector

    def in_edges(self,adj):
        output=0
        for i in self.not_populars:
            adj_popular = [j for j in adj[i] if self.vector[j]==1]
            output+=len(adj_popular)
        return output
    
    def evaluate(self,adj):
        in_edges=self.in_edges(adj)
        if in_edges==0:
            return self.Popular_size
        return 0
            
    def mutate(self,adj):
        new_populars=[]
        for i in self.populars:
            in_edges=[j for j in adj[:,i] if self.vector[j]==0]
            if len(in_edges) > 0 :
                self.vector[i]=0
                self.not_populars.append(i)
            else: new_populars.append(i)

        #index= random.randrange(len(self.vector))
        #vector=[i for i in self.vector]
        #vector[index]=1-self.vector[index]
        #return Solution(vector)

    def evaluate_1(self,adj,k):
        output=0
        #Contando la cantidad de personas en  el grupo de no populares a los cuales los odia al menos alguien del grupo de no populares  
        for i in self.populars:
            for j in adj[:][i]:
                if self.vector[j]==1:
                    output+=1
                    break
        return output

### Crear una solución random

In [ ]:
def generate_random_solution(all_indexes,vertex_total,k): 
    vector=np.zeros(vertex_total)
    populars_total=random.randrange(k+1)
    populars=set(random.sample(all_indexes,populars_total))
    for i in range(populars):
        vector[i]=1
    return Solution(populars,vector)

### 1 - Crear la primera población

In [ ]:
def initialize_population(pop_size,vertex_total,k):
    population_set = []
    all_indexes = [i for i in range(vertex_total) ]
    for i in range(pop_size):
        sol_i = generate_random_solution(all_indexes,vertex_total,k)
        population_set.append(sol_i)
        
    return np.array(population_set)

### 2 - Evaluación

In [ ]:
def fitness_eval(solution:Solution,adj):
    return solution.evaluate(adj)
                                     

In [ ]:
def get_all_fitnes(population_set, pop_size):
    fitnes_list = np.zeros(pop_size)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(pop_size):
        fitnes_list[i] = fitness_eval(population_set[i])

    return fitnes_list

### 3 - Selección

Crear dos listas de progenitores del tamaño de la población que indican que los progenitores en igual índice de ambas listas van a ser apareados.

In [ ]:


def progenitor_selection(population_set,fitnes_list,k):

    #for each pair of solutions get which would be the size of popular group if they get merge
    new_sizes={ i:[] for i in range(k+1)}
    for i,solution1 in enumerate(population_set):
        if i<len(population_set)-1:
            for j,solution2 in enumerate(population_set,i+1):
                p=fitnes_list[i]
                q=fitnes_list[j]
                if p > 0 and q > 0: 
                    total = len(solution1.populars | solution2.populars)
                    if total <=k:
                        new_sizes[total].append((i,j,True))
                    else: new_sizes[k].append((i,j,False))
                else: new_sizes[0].append((i,j,False))
                     
    progenitor_list_a = []
    progenitor_list_b = []
    include_all = []
    for i in range(k,0,1):
        if len(progenitor_list_a < len(population_set)):
            for pair in new_sizes[i]:
                if len(progenitor_list_a < len(population_set)):
                    progenitor_list_a.append(pair[0])
                    progenitor_list_b.append(pair[1])
                    include_all.append(pair[2])
                
    return np.array([progenitor_list_a,progenitor_list_b,include_all])

### 4 - Variación

In [ ]:
def mate_progenitors(prog_a, prog_b,include_all,k):
    new_populars={}
    if include_all: new_populars=prog_a.populars | prog_b.populars
    else: new_populars = random.choices(prog_a.populars | prog_b.populars,k=k)
    return Solution(new_populars)
    
    
def mate_population(progenitor_list,k):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b, include_all = progenitor_list[0][i], progenitor_list[1][i], progenitor_list[2][i]
        offspring = mate_progenitors(prog_a, prog_b,include_all,k)
        new_population_set.append(offspring)
        
    return new_population_set

In [ ]:
def mutate_offspring(offspring:Solution, mutation_rate):
    return offspring.mutate
    
def mutate_population(new_population_set, mutation_rate):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring, mutation_rate))
    return np.array(mutated_pop)

### 5 - Criterio de parada

In [ ]:
def stop_criterion(t):
    return t >= 10000 

In [ ]:
def evolutionary_algorithm(adj,k,pop_size = 100, mutation_rate = 0.3):  
    populations = []
    best_solution = [-1,-1,np.array([])] #[iteration, fitness, solution]
    
    populations.append(initialize_population(pop_size))
    t = 0
    
    while not stop_criterion(t):
        fitnes_list = get_all_fitnes(populations[t], pop_size)
        if t%100==0: print(t,  "fitness_mean: ", fitnes_list.mean(), "best_solution: ", best_solution)
            
        #Saving the best solution
        current_max = fitnes_list.max()
        if current_max > best_solution[1]:
            best_solution[0] = t
            best_solution[1] =current_max
            best_solution[2] = np.array(populations[t])[current_max == fitnes_list]  #arreglar!!!!!!!!!!!!!!!!!!!!
        
            if current_max==k: return best_solution

        progenitor_list = progenitor_selection(populations[t],fitnes_list)
        new_population_set = mate_population(progenitor_list)
        mutated_pop = mutate_population(new_population_set, mutation_rate)
        t = t+1
        
        populations.append(mutated_pop)
    
    return best_solution


